In [2]:
!pip install trtokenizer
!pip install TurkishStemmer

In [3]:
import pandas as pd
import re
import string
from trtokenizer.tr_tokenizer import SentenceTokenizer
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from snowballstemmer import stemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from gensim.models import Word2Vec
from TurkishStemmer import TurkishStemmer
import spacy
from spacy.lang.tr.stop_words import STOP_WORDS




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
df = pd.read_excel('tweetData.xlsx')

In [5]:
df = df.dropna()

In [6]:
df.shape

(1976, 2)

In [7]:
def remove_special_characters(text):
    try:
      text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
      return text
    except:
        pass

def to_lowercase(text):
    try:
      return text.lower()
    except:
        pass

def remove_extra_spaces(text):
    try:
      return  ' '.join(text.split())
    except:
        pass

def remove_html_tags(text):
    try:
      clean = re.compile('<.*?>')
      return re.sub(clean, '', text)
    except:
        pass

def remove_urls(text):
    try:
      return re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    except:
        pass

def remove_numbers(text):
    try:
      return re.sub(r'\d+', '', text)
    except:
        pass

def remove_punctuation(text):
    try:
      translator = str.maketrans('', '', string.punctuation)
      return text.translate(translator)
    except:
      pass

In [8]:
df['Tweets'] = df['Tweets'].apply(to_lowercase).apply(remove_extra_spaces).apply(remove_punctuation).apply(remove_html_tags).apply(remove_urls).apply(remove_numbers)

In [9]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df.Tweets.loc[15595:15599]

15595    kimmiş lan rahatsız olacak söylesin infaz edel...
15596                              i̇nstasina infaz edelim
15597    üstad latin abd ülkelerinde denediler direk ka...
15598    bu iki oevladının açık adreslerini verin infaz...
15599     kıbrısın tamamı alınmalı   ada diye bize yutt...
Name: Tweets, dtype: object

In [10]:
df.head()

,Tweets,Etiket
0,aynı bayrak aynı göğün altındayız biriz berabe...,1.0
1,çok istekli olan bu erkekler lütfen erkek çoc...,1.0
2,kurtlerle komşumuz alrabayiz beraberiz siz yal...,2.0
3,müslümanın seni beni ırkı olmaz dini olur kims...,1.0
4,soramazsına hoşgeldiniz bu gün bilge ile berab...,2.0


In [11]:


# Cümle ve kelime ayrıştırıcı işlevler
def trtokenizer_sentence(text):
    try:
        return sent_tokenize(text, language='turkish')
    except Exception as e:
        print(f"Sentence tokenization error: {e}")
        return []

def trtokenizer_word(text):
    try:
        return word_tokenize(text, language='turkish')
    except Exception as e:
        print(f"Word tokenization error: {e}")
        return []


def remove_stopwords_spacy(text):
    turkish_stopwords = STOP_WORDS
    cleaned = [word for word in text if word not in turkish_stopwords]
    return cleaned
def turkishstemmer(text):
  stemmer = TurkishStemmer()
  # Stem işlemi
  stemmed_words = [stemmer.stem(word) for word in text]
  return ' '.join(stemmed_words)




# Cümle ve kelime ayrıştırma işlemlerini DataFrame'e uygulama
df['sentences'] = df['Tweets'].apply(trtokenizer_sentence)

# Kelimeleri çıkarmak için her cümleyi kelimelere böler ve tüm kelimeleri düz bir liste olarak toplar
df['words'] = df['sentences'].apply(lambda sentences: [word for sentence in sentences for word in trtokenizer_word(sentence)])

# Stop kelimeleri temizleme işlemini ve birleştirme işlemini DataFrame'e uygulama
df['cleaned_words'] = df['words'].apply(remove_stopwords_spacy)
df['lemmatized_words'] = df['cleaned_words'].apply(turkishstemmer)

# Sonuçları yazdırma
print(df.head())




                                              Tweets  Etiket  \
0  aynı bayrak aynı göğün altındayız biriz berabe...     1.0   
1  çok istekli olan bu erkekler  lütfen erkek çoc...     1.0   
2  kurtlerle komşumuz alrabayiz beraberiz siz yal...     2.0   
3  müslümanın seni beni ırkı olmaz dini olur kims...     1.0   
4  soramazsına hoşgeldiniz bu gün bilge ile berab...     2.0   

                                           sentences  \
0  [aynı bayrak aynı göğün altındayız biriz berab...   
1  [çok istekli olan bu erkekler  lütfen erkek ço...   
2  [kurtlerle komşumuz alrabayiz beraberiz siz ya...   
3  [müslümanın seni beni ırkı olmaz dini olur kim...   
4  [soramazsına hoşgeldiniz bu gün bilge ile bera...   

                                               words  \
0  [aynı, bayrak, aynı, göğün, altındayız, biriz,...   
1  [çok, istekli, olan, bu, erkekler, lütfen, erk...   
2  [kurtlerle, komşumuz, alrabayiz, beraberiz, si...   
3  [müslümanın, seni, beni, ırkı, olmaz, dini, ol...  

In [12]:
# Kelimeleri tokenize et
tokenize_metni = [metin.lower().split() for metin in df['Tweets']]

# Word2Vec modelini eğit
word2vec_model = Word2Vec(sentences=tokenize_metni, vector_size=100, window=5, min_count=1, workers=4)

# Her metin için ortalama kelime vektörlerini hesapla
def ortalama_vektor(metin):
    kelimeler = metin.lower().split()
    vektörler = [word2vec_model.wv[kelime] for kelime in kelimeler if kelime in word2vec_model.wv]
    return sum(vektörler) / len(vektörler) if vektörler else [0]*100

df['word2vec_vektör'] = df['Tweets'].apply(ortalama_vektor)

print(df['word2vec_vektör'])

0        [-0.0006315285, 0.0033361074, 0.003708357, -0....
1        [-0.0032319494, 0.009215738, 0.0015737953, 0.0...
2        [-0.0011103889, 0.005456079, -0.0002666204, -0...
3        [-0.0039284914, 0.009343298, 0.0027607062, 0.0...
4        [-0.004111429, 0.0069497796, 0.0057134787, 0.0...
5        [-0.00092673645, 0.004137669, 0.00029781775, -...
6        [-0.00324949, 0.010974731, 0.0015393664, 0.001...
7        [-0.0002503334, 0.006313625, 0.0022002163, -0....
8        [-0.0017441299, 0.012320818, -0.00033566804, 0...
9        [-0.0021507086, 0.008174099, 0.0009368965, 0.0...
10       [-0.0024864788, 0.0076900166, 0.00080746715, 0...
11       [-0.004264857, 0.006133479, 0.0026120832, 0.00...
12       [-0.0049062986, 0.010543617, 0.0014850511, 0.0...
13       [-0.0065027834, 0.0066494066, 0.0038135976, 0....
14       [-0.0012367454, 0.0072927307, 0.001972299, -0....
15       [-0.0025780452, 0.005954157, 0.0022405111, 0.0...
16       [-0.0026030932, 0.00871836, 0.00075826026, 0.0.